In [1]:
import pandas as pd
import numpy as np

# Financial Well-Being Survey: Data Cleaning
#### Consumer Financial Protection Bureau (CFPB), 2017


In [2]:
df = pd.read_csv("./NFWBS_PUF_2016_data.csv")
df.head()

,PUF_ID,sample,fpl,SWB_1,SWB_2,SWB_3,FWBscore,FWB1_1,FWB1_2,FWB1_3,...,PPMSACAT,PPREG4,PPREG9,PPT01,PPT25,PPT612,PPT1317,PPT18OV,PCTLT200FPL,finalwt
0,10350,2,3,5,5,6,55,3,3,3,...,1,4,8,0,0,0,0,1,0,0.367292
1,7740,1,3,6,6,6,51,2,2,3,...,1,2,3,0,0,0,0,2,0,1.327561
2,13699,1,3,4,3,4,49,3,3,3,...,1,4,9,0,0,0,1,2,1,0.835156
3,7267,1,3,6,6,6,49,3,3,3,...,1,3,7,0,0,0,0,1,0,1.410871
4,7375,1,3,4,4,4,49,3,3,3,...,1,2,4,0,0,1,0,4,1,4.260668


## Getting familiar with the data set

In [3]:
list(df.columns)

['PUF_ID',
 'sample',
 'fpl',
 'SWB_1',
 'SWB_2',
 'SWB_3',
 'FWBscore',
 'FWB1_1',
 'FWB1_2',
 'FWB1_3',
 'FWB1_4',
 'FWB1_5',
 'FWB1_6',
 'FWB2_1',
 'FWB2_2',
 'FWB2_3',
 'FWB2_4',
 'FSscore',
 'FS1_1',
 'FS1_2',
 'FS1_3',
 'FS1_4',
 'FS1_5',
 'FS1_6',
 'FS1_7',
 'FS2_1',
 'FS2_2',
 'FS2_3',
 'SUBKNOWL1',
 'ACT1_1',
 'ACT1_2',
 'FINGOALS',
 'PROPPLAN_1',
 'PROPPLAN_2',
 'PROPPLAN_3',
 'PROPPLAN_4',
 'MANAGE1_1',
 'MANAGE1_2',
 'MANAGE1_3',
 'MANAGE1_4',
 'SAVEHABIT',
 'FRUGALITY',
 'AUTOMATED_1',
 'AUTOMATED_2',
 'ASK1_1',
 'ASK1_2',
 'SUBNUMERACY2',
 'SUBNUMERACY1',
 'CHANGEABLE',
 'GOALCONF',
 'LMscore',
 'FINKNOWL1',
 'FINKNOWL2',
 'FINKNOWL3',
 'FK1correct',
 'FK2correct',
 'FK3correct',
 'KHscore',
 'KHKNOWL1',
 'KHKNOWL2',
 'KHKNOWL3',
 'KHKNOWL4',
 'KHKNOWL5',
 'KHKNOWL6',
 'KHKNOWL7',
 'KHKNOWL8',
 'KHKNOWL9',
 'KH1correct',
 'KH2correct',
 'KH3correct',
 'KH4correct',
 'KH5correct',
 'KH6correct',
 'KH7correct',
 'KH8correct',
 'KH9correct',
 'ENDSMEET',
 'HOUSING',
 'LIVING

### Looking at Nulls

In [4]:
df.isnull().any()

PUF_ID         False
sample         False
fpl            False
SWB_1          False
SWB_2          False
               ...  
PPT612         False
PPT1317        False
PPT18OV        False
PCTLT200FPL    False
finalwt        False
Length: 217, dtype: bool

In [5]:
list(df.isnull().any())

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,


We can see that none of the columns contain any NULL values, meaning that we are **not** missing any data. However, it is important to note that our dataset is a survey, so most of our columns are survey questions and some of the allowed responses include "Refused" (represented by the value -1). We also have "Response not written to dataset" (represented by the value -4). In order to get the response rate for the survey questions, we will create a copy of the dataframe and replace these values with None/Null, given that they don't answer the survey question asked. Keep in mind this is just a check; we are not getting rid of these values at this point.

### Getting survey response rates

In [6]:
nonresponse_values = [-1, -4]
temp_df = df.copy()
cols = temp_df.columns
for col in cols:
    col_values = temp_df[col]
    temp_df[col] = np.where(col_values.isin(nonresponse_values),None,temp_df[col])
temp_df.head()

,PUF_ID,sample,fpl,SWB_1,SWB_2,SWB_3,FWBscore,FWB1_1,FWB1_2,FWB1_3,...,PPMSACAT,PPREG4,PPREG9,PPT01,PPT25,PPT612,PPT1317,PPT18OV,PCTLT200FPL,finalwt
0,10350,2,3,5,5,6,55,3,3,3,...,1,4,8,0,0,0,0,1,0,0.367292
1,7740,1,3,6,6,6,51,2,2,3,...,1,2,3,0,0,0,0,2,0,1.32756
2,13699,1,3,4,3,4,49,3,3,3,...,1,4,9,0,0,0,1,2,1,0.835156
3,7267,1,3,6,6,6,49,3,3,3,...,1,3,7,0,0,0,0,1,0,1.41087
4,7375,1,3,4,4,4,49,3,3,3,...,1,2,4,0,0,1,0,4,1,4.26067


In [7]:
temp_df.isnull().any()

PUF_ID         False
sample         False
fpl            False
SWB_1           True
SWB_2           True
               ...  
PPT612         False
PPT1317        False
PPT18OV        False
PCTLT200FPL    False
finalwt        False
Length: 217, dtype: bool

In [8]:
response_rates = {}
for col in cols:
    valid_value_count = temp_df[col].count()
    total_rows = len(temp_df[col])
    res_rate = round(valid_value_count/total_rows,2)
    response_rates[col] = res_rate
response_rates

{'PUF_ID': 1.0,
 'sample': 1.0,
 'fpl': 1.0,
 'SWB_1': 1.0,
 'SWB_2': 0.99,
 'SWB_3': 0.99,
 'FWBscore': 1.0,
 'FWB1_1': 1.0,
 'FWB1_2': 1.0,
 'FWB1_3': 1.0,
 'FWB1_4': 1.0,
 'FWB1_5': 1.0,
 'FWB1_6': 1.0,
 'FWB2_1': 1.0,
 'FWB2_2': 1.0,
 'FWB2_3': 1.0,
 'FWB2_4': 1.0,
 'FSscore': 1.0,
 'FS1_1': 1.0,
 'FS1_2': 1.0,
 'FS1_3': 1.0,
 'FS1_4': 1.0,
 'FS1_5': 1.0,
 'FS1_6': 1.0,
 'FS1_7': 1.0,
 'FS2_1': 1.0,
 'FS2_2': 1.0,
 'FS2_3': 1.0,
 'SUBKNOWL1': 0.99,
 'ACT1_1': 1.0,
 'ACT1_2': 1.0,
 'FINGOALS': 0.99,
 'PROPPLAN_1': 1.0,
 'PROPPLAN_2': 1.0,
 'PROPPLAN_3': 1.0,
 'PROPPLAN_4': 1.0,
 'MANAGE1_1': 1.0,
 'MANAGE1_2': 1.0,
 'MANAGE1_3': 1.0,
 'MANAGE1_4': 1.0,
 'SAVEHABIT': 1.0,
 'FRUGALITY': 1.0,
 'AUTOMATED_1': 0.99,
 'AUTOMATED_2': 0.98,
 'ASK1_1': 1.0,
 'ASK1_2': 1.0,
 'SUBNUMERACY2': 0.99,
 'SUBNUMERACY1': 0.99,
 'CHANGEABLE': 0.99,
 'GOALCONF': 1.0,
 'LMscore': 1.0,
 'FINKNOWL1': 1.0,
 'FINKNOWL2': 0.99,
 'FINKNOWL3': 0.99,
 'FK1correct': 1.0,
 'FK2correct': 1.0,
 'FK3correct': 1.0,
 

In [9]:
min(response_rates.values())

0.9

We can see the that minimum response rate for all the survey variables is 90%, which is pretty high. We may be able to keep all of the selected columns without worrying that values available don't answer the survey questions, given that they are expected to have a 90% response rate or higher.

### Looking at non-responses further

In [10]:
# "HOUSING":{
#   -1: "Refused",
#   1: "I own my home",
#   2: "I rent",
#   3: "I do not currently own or rent"

df['HOUSING'].value_counts()

 1    4164
 2    1643
 3     545
-1      42
Name: HOUSING, dtype: int64

In [11]:
# "PPETHM":{
#   1: "White, Non-Hispanic",
#   2: "Black, Non-Hispanic",
#   3: "Other, Non-Hispanic",
#   4: "Hispanic"
# },
df[df['HOUSING'] == -1].groupby(['PPETHM']).count().reset_index().loc[:,['PPETHM','HOUSING']].rename(columns={'PPETHM':'Ethicity/Race','HOUSING':'count_refused_to_answer'})


,Ethicity/Race,count_refused_to_answer
0,1,29
1,2,5
2,3,2
3,4,6


As previously mentioned, the survey has values "Refused" (represented by the value -1) and "Response not written to dataset" (represented by the value -4) in the dataset. Even though these values don't necessary directly answer the questions being asked in the survey, they still give us data. They essentially have the potential to provide additional insights, like what participants didn't answer certain questions. It will be interesting to investigate and see those patterns.

## Getting columns of interest

In [12]:
# Columns that most relate to our problem statement
columns_of_interest = [
    'PUF_ID',
    'agecat',
    'PPEDUC',
    'PPETHM',
    'PPGENDER',
    'PPINCIMP',
    'PPHHSIZE',
    'fpl',
    'PPREG4',
    'PPREG9',
    'FWB1_1',
    'FWB1_5',
    'FS2_3',
    'SUBKNOWL1',
    'ACT1_2',
    'PROPPLAN_3',
    'PROPPLAN_1',
    'MANAGE1_2',
    'MANAGE1_1',
    'MANAGE1_3',
    'SAVEHABIT',
    'AUTOMATED_1',
    'AUTOMATED_2',
    'SAVINGSRANGES',
    'HOUSING',
    'HOUSERANGES',
    'MORTGAGE',
    'PRODHAVE_8',
    'PRODUSE_1',
    'PRODUSE_2',
    'FWBscore',
    'FSscore'
]

The problem statement we are hoping to address is: How does financial literacy education impact a marginalized population’s debt to income rate and home ownership ability?

The columns listed above are variables that provide data to answer/give insights to our problem statement. These variables all relate to one of the following: Demographics, Income, Location, Financial Well-Being, Financial Skill, Financial Behavior, Financial Planning, Expenses, Debt, Credit, Savings, Home Ownership.

In [13]:
filtered_df = df.loc[:,columns_of_interest]
filtered_df.head()

,PUF_ID,agecat,PPEDUC,PPETHM,PPGENDER,PPINCIMP,PPHHSIZE,fpl,PPREG4,PPREG9,...,AUTOMATED_2,SAVINGSRANGES,HOUSING,HOUSERANGES,MORTGAGE,PRODHAVE_8,PRODUSE_1,PRODUSE_2,FWBscore,FSscore
0,10350,8,4,1,1,7,1,3,4,8,...,0,6,1,4,2,0,0,0,55,44
1,7740,3,2,1,1,6,2,3,2,3,...,0,2,1,4,2,0,0,0,51,43
2,13699,3,3,2,1,6,3,3,4,9,...,1,4,1,3,2,0,0,0,49,42
3,7267,3,2,1,1,8,1,3,3,7,...,-1,-1,-1,99,-2,0,0,0,49,42
4,7375,2,2,3,1,7,5,3,2,4,...,1,98,2,2,-2,0,0,1,49,42


## For reference (when doing Exploratory Data Analysis)

In [14]:
# Value mapping to actual responses; might come in handy when doing EDA
value_map = {
    "agecat":{
      1: "18-24",
      2: "25-34",
      3: "35-44",
      4: "45-54",
      5: "55-61",
      6: "62-69",
      7: "70-74",
      8: "75+"
    },
    "PPEDUC":{
      1: "Less than high school",
      2: "High school degree/GED",
      3: "Some college/Associate",
      4: "Bachelors degree",
      5: "Graduate/professional degree"
    },
    "PPETHM":{
      1: "White, Non-Hispanic",
      2: "Black, Non-Hispanic",
      3: "Other, Non-Hispanic",
      4: "Hispanic"
    },
    "PPGENDER":{
      1: "Male",
      2: "Female"
    },
    "PPINCIMP":{
      1: "Less than $20,000",
      2: "$20,000 to $29,999",
      3: "$30,000 to $39,999",
      4: "$40,000 to $49,999",
      5: "$50,000 to $59,999",
      6: "$60,000 to $74,999",
      7: "$75,000 to $99,999",
      8: "$100,000 to $149,999",
      9: "$150,000 or more"
    },
    "PPHHSIZE":{
      1: "1",
      2: "2",
      3: "3",
      4: "4",
      5: "5+"
    },
    "fpl":{
      1: "<100% FPL",
      2: "100%-199% FPL",
      3: "200%+ FPL"
    },
    "PPREG4":{
      1: "Northeast",
      2: "Midwest",
      3: "South",
      4: "West"
    },
    "PPREG9":{
      1: "New England",
      2: "Mid-Atlantic",
      3: "East-North Central",
      4: "West-North Central",
      5: "South Atlantic",
      6: "East-South Central",
      7: "West-South Central",
      8: "Mountain",
      9: "Pacific"
    },
    "FWB1_1":{
      -4: "Response not written to database",
      -1: "Refused",
      1: "Not at all",
      2: "Very little",
      3: "Somewhat",
      4: "Very well",
      5: "Completely"
    },
    "FWB1_5":{
      -4: "Response not written to database",
      -1: "Refused",
      1: "Not at all",
      2: "Very little",
      3: "Somewhat",
      4: "Very well",
      5: "Completely"
    },
    "FS2_3":{
      -1: "Refused",
      1: "Never",
      2: "Rarely",
      3: "Sometimes",
      4: "Often",
      5: "Always"
    },
    "SUBKNOWL1":{
      -1: "Refused",
      1: "1 - Very low",
      2: "2",
      3: "3",
      4: "4",
      5: "5",
      6: "6",
      7: "7 - Very high"
    }
}

In [15]:
# Column mapping to variable label; might come in handy when doing EDA
col_map = {
    'agecat':'Age Group',
    'PPEDUC':'Education (Highest Degree)',
    'PPETHM':'Race/Ethnicity',
    'PPGENDER':'Gender',
    'PPINCIMP':'Household Income',
    'PPHHSIZE':'Household Size',
    'fpl':'Federal Poverty Level',
    'PPREG4':'Census Region',
    'PPREG9':'Census Division',
    'FWB1_1':'I could handle a major unexpected expense.',
    'FWB1_5':'I am just getting by financially',
    'FS2_3':'I struggle to understand financial information',
    'SUBKNOWL1':'How would you assess your overall financial knowledge?',
    'ACT1_2':'I follow-through on financial goals I set for myself.',
    'PROPPLAN_3':'I set financial goals for what I want to achieve with my money',
    'PROPPLAN_1':'I consult my budget to see how much money I have left',
    'MANAGE1_2':'Stayed within your budget or spending plan',
    'MANAGE1_1':'Paid all your bills on time',
    'MANAGE1_3':'Paid off credit card balance in full',
    'SAVEHABIT':'Putting money into savings is a habit for me',
    'AUTOMATED_1':'A Retirement Savings Account',
    'AUTOMATED_2':'A Non-Retirement Savings Account',
    'SAVINGSRANGES':'How much money do you have in savings today?',
    'HOUSING':'Which one of the following best describes your housing situation?',
    'HOUSERANGES':'About how much do you pay for your home each month?',
    'MORTGAGE':'What do you owe on your home?',
    'PRODHAVE_8':'Student/Education Loan (for yourself or someone else)',
    'PRODUSE_1':'Used payday loan or cash advance loan',
    'PRODUSE_2':'Used pawn loan or auto title loan',
    'FWBscore':'Financial Well-Being Scale score',
    'FSscore':'Financial Skill Scale score'
}

In [16]:
filtered_df

,PUF_ID,agecat,PPEDUC,PPETHM,PPGENDER,PPINCIMP,PPHHSIZE,fpl,PPREG4,PPREG9,...,AUTOMATED_2,SAVINGSRANGES,HOUSING,HOUSERANGES,MORTGAGE,PRODHAVE_8,PRODUSE_1,PRODUSE_2,FWBscore,FSscore
0,10350,8,4,1,1,7,1,3,4,8,...,0,6,1,4,2,0,0,0,55,44
1,7740,3,2,1,1,6,2,3,2,3,...,0,2,1,4,2,0,0,0,51,43
2,13699,3,3,2,1,6,3,3,4,9,...,1,4,1,3,2,0,0,0,49,42
3,7267,3,2,1,1,8,1,3,3,7,...,-1,-1,-1,99,-2,0,0,0,49,42
4,7375,2,2,3,1,7,5,3,2,4,...,1,98,2,2,-2,0,0,1,49,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6389,11220,7,5,2,2,9,3,3,2,3,...,1,3,1,6,2,0,0,0,61,47
6390,13118,6,2,2,1,3,3,2,3,6,...,7,99,1,99,99,0,0,0,59,59
6391,8709,2,5,1,2,9,2,3,1,2,...,1,6,2,5,-2,1,0,0,59,51
6392,8515,2,2,4,1,4,2,3,4,9,...,7,99,2,5,-2,0,0,0,46,54


In [17]:
filtered_df =filtered_df.replace(value_map)

In [18]:
filtered_df

,PUF_ID,agecat,PPEDUC,PPETHM,PPGENDER,PPINCIMP,PPHHSIZE,fpl,PPREG4,PPREG9,...,AUTOMATED_2,SAVINGSRANGES,HOUSING,HOUSERANGES,MORTGAGE,PRODHAVE_8,PRODUSE_1,PRODUSE_2,FWBscore,FSscore
0,10350,75+,Bachelors degree,"White, Non-Hispanic",Male,"$75,000 to $99,999",1,200%+ FPL,West,Mountain,...,0,6,1,4,2,0,0,0,55,44
1,7740,35-44,High school degree/GED,"White, Non-Hispanic",Male,"$60,000 to $74,999",2,200%+ FPL,Midwest,East-North Central,...,0,2,1,4,2,0,0,0,51,43
2,13699,35-44,Some college/Associate,"Black, Non-Hispanic",Male,"$60,000 to $74,999",3,200%+ FPL,West,Pacific,...,1,4,1,3,2,0,0,0,49,42
3,7267,35-44,High school degree/GED,"White, Non-Hispanic",Male,"$100,000 to $149,999",1,200%+ FPL,South,West-South Central,...,-1,-1,-1,99,-2,0,0,0,49,42
4,7375,25-34,High school degree/GED,"Other, Non-Hispanic",Male,"$75,000 to $99,999",5+,200%+ FPL,Midwest,West-North Central,...,1,98,2,2,-2,0,0,1,49,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6389,11220,70-74,Graduate/professional degree,"Black, Non-Hispanic",Female,"$150,000 or more",3,200%+ FPL,Midwest,East-North Central,...,1,3,1,6,2,0,0,0,61,47
6390,13118,62-69,High school degree/GED,"Black, Non-Hispanic",Male,"$30,000 to $39,999",3,100%-199% FPL,South,East-South Central,...,7,99,1,99,99,0,0,0,59,59
6391,8709,25-34,Graduate/professional degree,"White, Non-Hispanic",Female,"$150,000 or more",2,200%+ FPL,Northeast,Mid-Atlantic,...,1,6,2,5,-2,1,0,0,59,51
6392,8515,25-34,High school degree/GED,Hispanic,Male,"$40,000 to $49,999",2,200%+ FPL,West,Pacific,...,7,99,2,5,-2,0,0,0,46,54


In [19]:
filtered_df = filtered_df.rename(columns = col_map)

In [20]:
filtered_df

,PUF_ID,Age Group,Education (Highest Degree),Race/Ethnicity,Gender,Household Income,Household Size,Federal Poverty Level,Census Region,Census Division,...,A Non-Retirement Savings Account,How much money do you have in savings today?,Which one of the following best describes your housing situation?,About how much do you pay for your home each month?,What do you owe on your home?,Student/Education Loan (for yourself or someone else),Used payday loan or cash advance loan,Used pawn loan or auto title loan,Financial Well-Being Scale score,Financial Skill Scale score
0,10350,75+,Bachelors degree,"White, Non-Hispanic",Male,"$75,000 to $99,999",1,200%+ FPL,West,Mountain,...,0,6,1,4,2,0,0,0,55,44
1,7740,35-44,High school degree/GED,"White, Non-Hispanic",Male,"$60,000 to $74,999",2,200%+ FPL,Midwest,East-North Central,...,0,2,1,4,2,0,0,0,51,43
2,13699,35-44,Some college/Associate,"Black, Non-Hispanic",Male,"$60,000 to $74,999",3,200%+ FPL,West,Pacific,...,1,4,1,3,2,0,0,0,49,42
3,7267,35-44,High school degree/GED,"White, Non-Hispanic",Male,"$100,000 to $149,999",1,200%+ FPL,South,West-South Central,...,-1,-1,-1,99,-2,0,0,0,49,42
4,7375,25-34,High school degree/GED,"Other, Non-Hispanic",Male,"$75,000 to $99,999",5+,200%+ FPL,Midwest,West-North Central,...,1,98,2,2,-2,0,0,1,49,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6389,11220,70-74,Graduate/professional degree,"Black, Non-Hispanic",Female,"$150,000 or more",3,200%+ FPL,Midwest,East-North Central,...,1,3,1,6,2,0,0,0,61,47
6390,13118,62-69,High school degree/GED,"Black, Non-Hispanic",Male,"$30,000 to $39,999",3,100%-199% FPL,South,East-South Central,...,7,99,1,99,99,0,0,0,59,59
6391,8709,25-34,Graduate/professional degree,"White, Non-Hispanic",Female,"$150,000 or more",2,200%+ FPL,Northeast,Mid-Atlantic,...,1,6,2,5,-2,1,0,0,59,51
6392,8515,25-34,High school degree/GED,Hispanic,Male,"$40,000 to $49,999",2,200%+ FPL,West,Pacific,...,7,99,2,5,-2,0,0,0,46,54


In [23]:
len(filtered_df.columns)

32

In [24]:
filtered_df.to_csv('CFPB_FILTED_MAPPED.csv')